In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI()

In [4]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        Here is what I know:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        Here is what I know:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)

example_msg_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)

In [9]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that provides information about countries.",
        ),
        example_msg_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "United Kingdom"})

AIMessage(content='Here is what I know:\n        Capital: London\n        Language: English\n        Food: Fish and Chips\n        Currency: Pound Sterling')

In [10]:
from langchain.prompts import load_prompt

prompt = load_prompt("../prompts/prompt.json")

prompt.format(country="Korea")

'What is the capital of Korea?'

In [11]:
prompt = load_prompt("../prompts/prompt.yaml")

prompt.format(country="Korea")

'What currency is used in Korea?'

In [12]:
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}.
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "pirate",
        "example_question": "Where are you now?",
        "example_answer": "Arrrrg... That is a secret, Arg arg!",
        "question": "What is your favorite treasure?"
    }
)

AIMessage(content="Arrrrg matey! Me favorite treasure be the legendary Lost Pearl of the Seven Seas! Shrouded in mystery and guarded by fierce creatures, it be a treasure worth riskin' life and limb for, arrr!")

In [13]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    ans = chat.predict("How can I know how much this question costs?")
    print(ans)
    print(usage)

There is no cost associated with asking questions to this AI assistant. It is free to use and ask as many questions as you like.
Tokens Used: 44
	Prompt Tokens: 17
	Completion Tokens: 27
Successful Requests: 1
Total Cost (USD): $7.950000000000001e-05
